## Preparing the docker images for Darknet (yolo)

In [ ]:
repo_name=<<YOUR ECR URL>> # ie. "<<your account id here>>.dkr.ecr.us-east-1.amazonaws.com"
region_name=<<SERVICE REGION>> # ie. 'us-east-1'


darknet_gpu="darknet:gpu-1.0"
darknet_cpu="darknet:cpu-1.0"

In [ ]:
def prepare_image(dockerfile, repo_name, tag_name):
    !cd Docker && sudo docker build -f $dockerfile -t $tag_name .
    !cd Docker && sudo docker tag $tag_name $repo_name/$tag_name
    login=!aws ecr get-login --no-include-email
    login=login[0]
    !eval "sudo $login"
    !sudo docker push $repo_name/$tag_name

### First the GPU version

In [ ]:
prepare_image("Dockerfile.gpu", repo_name, darknet_gpu)

### Now, the CPU version

In [ ]:
prepare_image("Dockerfile.cpu", repo_name, darknet_cpu)

## Now, let's deploy a pre-trained yolo model, for Object Detection

In [ ]:
%%time
import boto3
import re
import os
import time

from time import gmtime, strftime
from sagemaker import get_execution_role

import sagemaker

# Get the current Sagemaker session
sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()

# 1. Obtaining the role you already configured for Sagemaker when you setup
# your Instance notebook (https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
role = get_execution_role()

# 2. The S3 Bucket that will store the dataset and the trained model
# It was already defined above, while we uploaded the RecordIO files to the S3 bucket.

# 3. Select the correct Docker image with the Image Classification algorithm
containers = {region_name: '%s/%s' % ( repo_name, darknet_gpu) }
training_image = containers[boto3.Session().region_name]
print(training_image)

### Here we will download the pre-trained model and create a package

In [ ]:
%%bash

mkdir -p /tmp/model
cd /tmp/model

curl https://pjreddie.com/media/files/yolov3.weights -o model.weights
curl https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/coco.data -o model.data
curl https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -o model.cfg

tar -czvf ../model.tar.gz .

### Upload the model

In [ ]:
model_data = sagemaker_session.upload_data(path='/tmp/model.tar.gz', key_prefix='data/yolov3')
print(model_data)

### Cleaning the downloaded files

In [ ]:
!rm -rf /tmp/model.tar.gz /tmp/model

### Deploying the model/endpoint

In [ ]:
# Create an Mxnet Estimator
m = sagemaker.model.Model(image=training_image, model_data=model_data, role=role)

In [ ]:
%%time

# Publishes the model. It takes around 8mins
predictor = m.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

In [ ]:
endpoint_name = predictor.endpoint

## Testing

In [ ]:
%matplotlib inline

import cv2
import boto3
import matplotlib.pyplot as plt

from sagemaker.predictor import json_serializer, json_deserializer

In [ ]:
def predict(filename):
    img = open(filename, 'rb').read()
    sm = boto3.client('sagemaker-runtime')
    response = sm.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/x-image',
        Body=bytearray(img)
    )
    resp = json_deserializer(response['Body'], response['ContentType']) 
    return resp['detections']

In [ ]:
def render_bboxes(filename):
    bboxes = predict(filename)
    img = cv2.imread(filename)
    h,w,_ = img.shape
    
    thick = int((h + w) // 400)
    for i in range(len(bboxes)):
        category = bboxes[i][0]
        confidence = bboxes[i][1]
        bbox = bboxes[i][2]
        
        bW = int(bbox[2])
        bH = int(bbox[3])
        
        x1 = int(bbox[0]-bW/2)
        y1 = int(bbox[1]-bH/2)
        if x1 < 0: 
            x1 = 0 
        if y1 < 0:
            y1 = 0
        
        x2 = int(x1+bW)
        y2 = int(y1+bH)
    
        cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), thick)
        cv2.putText(img, "%s (%f)" %( category, confidence),(x1, y1), 0, 0.5, (0,0,255), thick//2)
        

    plt.figure(figsize=(12,8))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

### Let's get a sample and run it

In [ ]:
!curl https://qph.fs.quoracdn.net/main-qimg-99262df5d519ae1c5196f7626249583b -o /tmp/dog.jpg

In [ ]:
%%time
render_bboxes('/tmp/dog.jpg')

## Delete the endpoint

In [ ]:
import sagemaker
sagemaker.Session().delete_endpoint(endpoint_name)